In [ ]:
#importing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gensim
import nltk
import pickle
import re
import heapq
import spacy

from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix,  classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC  
from sklearn import preprocessing


In [ ]:
from Portal import *
# pd.set_option('display.width', 400)
# pd.set_option('display.max_columns', 10)


class CareersFuture(Portal):

    def __init__(self, name):
        super().__init__(name)

    def read_data(self):
        # Reads data
        df = pd.read_csv('mycareerfuture2020-09-15.csv', error_bad_lines=False)
        return df

    def clean_data(self):
        df = self.read_data()
        # df = df[df['Job Description'].str.contains('value')]
        df['Job Description'] = df['Job Description'].astype(str)
        df['Job Description'] = df['Job Description'].apply(lambda x: self.strip_html_tags(x))

        df['Industry'] = df['Industry'].astype(str)
        df['Industry'] = df['Industry'].apply(lambda x: self.str_to_literal(x))

        # df['Skills'] = df['Skills'].astype(str)

        df['Job Experience Required (years)'] = df['Job Experience Required (years)'].astype(int)

        df['Job Monthly Min Sal'] = df['Job Monthly Min Sal'].astype(int)
        df['Job Monthly Max Sal'] = df['Job Monthly Max Sal'].astype(int)

        return df

    def str_to_literal(self, text):
        try:
            # ast.literal_eval function converts takes in a string and converts it into a dictionary
            ls = super().str_to_literal(text)
            industry = []
            for i in ls:
                industry.append(i['category'])
            return industry
        except:
            pass

careersfuture = CareersFuture("careersfuture")


In [ ]:
cf = pd.DataFrame(careersfuture.clean_data())
cf['Industry_unlisted'] = cf['Industry'].apply(lambda x: ','.join(map(str, x)))
cf['Skills'] = cf['Skills'].apply(lambda x: x.replace("'", "") )

cf['Skills_unlisted'] = cf['Skills'].apply(lambda x: x.replace('[','').replace(']',''))
cf['Skills_unlisted'] = cf["Skills_unlisted"].apply(lambda a: a.lower())
cf_model = cf[["Job Title", "Job Description", "Industry_unlisted", "Skills_unlisted", "Job Experience Required (years)","Job Monthly Min Sal", "Job Monthly Max Sal" ]]


In [ ]:
cf_model

In [ ]:
cf_model[cf_model['Job Title'].str.contains('Cloud Operations Manage')]

In [ ]:
sf_sectors = pd.read_csv("sfw_sector.csv")

In [ ]:
sf_sectors['Skill Title'] = sf_sectors["Skill Title"].apply(lambda a: a.lower())

In [ ]:
sf_sectors

In [ ]:
pd.Series(cf_model['Skills_unlisted'][0].split(",")).apply(lambda x: x.strip()).tolist()

In [ ]:
sf_sectors['Job_Role_Replaced'] = sf_sectors['Job_Role_Replaced'].str.replace('head of ', '')

def clear_hierarchy(title):
    split = title.split()
    if (split[0] == 'assist') or (split[0]=='head'):
        final_title = " ".join(split[1:])
    else:
        final_title = title
    return final_title

sf_sectors['Job_Role_Replaced'] = sf_sectors['Job_Role_Replaced'].apply(lambda x: clear_hierarchy(x))
sf_sectors['Job_Role_Replaced'] = sf_sectors['Job_Role_Replaced'].str.replace('(specialist)', '')
sf_sectors['Job_Role_Replaced'] = sf_sectors['Job_Role_Replaced'].str.replace('specialist', '')

In [ ]:
sf_sectors = pd.read_csv('sf_model_final.csv')

In [ ]:
sf_sectors

In [ ]:
sf_sectors['Skill Title'] = sf_sectors['Skill Title'].apply(lambda x: x.replace("'", ""))
sf_sectors['Skills_unlisted'] = sf_sectors['Skill Title'].apply(lambda x: x.replace('[','').replace(']','').replace(",", " "))

In [ ]:
sf_sectors

In [ ]:
# X_train_sf, X_test_sf, y_train_sf, y_test_sf = train_test_split(sf_sectors['Skill Title'], 
#                                                     sf_sectors['Sector'], 
#                                                     test_size = 0.03, 
#                                                     shuffle = True, 
#                                                     stratify = sf_sectors['Sector'], 
#                                                     random_state = 3)

In [ ]:
transformer = Pipeline([('vect', CountVectorizer(max_features = 5000, stop_words = 'english', min_df = 3, max_df = 0.5)),
                ('tfidf', TfidfTransformer()),
                                
                ])
X_train_dtm = transformer.fit_transform(sf_sectors['Skills_unlisted'])

In [ ]:
#k_fold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 50)

In [ ]:
#param_grid = {'C': [0.1,1, 10], 'gamma': [1,0.1,0.01, 0.001],'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}

In [ ]:
# model = GridSearchCV(SVC(class_weight='balanced', probability = True),param_grid,refit=True,verbose=2, n_jobs = -1, cv = 3)

# for train_indices, test_indices in k_fold.split(sf_sectors['Skill Title'], sf_sectors['Sector']):
#     X_train, X_test = X_train_dtm[train_indices], X_train_dtm[test_indices]
#     y_train, y_test = sf_sectors['Sector'][train_indices], sf_sectors['Sector'][test_indices]
#     model.fit(X_train, y_train)
#     print("Best parameters set found on development set:")
#     print()
#     print(model.best_params_)
#     print()

#     y_true, y_pred = y_test, model.predict(X_test)
#     print(classification_report(y_true, y_pred))
#     print()



# # y_pred = logreg.predict(X_test_sf)

In [ ]:
#model.best_params_

In [ ]:
model = SVC(C = 1, gamma = 1, kernel = 'linear', class_weight='balanced', probability=True)

In [ ]:
model.fit(X_train_dtm, sf_sectors['Sector'])

In [ ]:
pickle.dump(transformer, open("tfidf.pickle", "wb"))
pickle.dump(model, open("model.sav", "wb"))

In [ ]:
vectorizer = pickle.load(open("tfidf.pickle", "rb"))
model = pickle.load(open('model.sav', 'rb'))
nlp = spacy.load("en_core_web_lg")

In [ ]:
def get_labels(data, column):
    
    le = preprocessing.LabelEncoder()
    for i in range(len(data)):
        data['Sector Label'] = le.fit_transform(data[column])
        values = sf_sectors[['Sector Label', column]].drop_duplicates().sort_values('Sector Label').reset_index().drop(columns = 'index')
        return values

In [ ]:
mydict = np.load('skills_dict.npy',allow_pickle=True) 

In [ ]:
def clear_list(role):
    
    return role.replace("['",'').replace("']",'')

In [ ]:
def replace_job_portal_skills(skills_list, mydict):
    new_list = []
    for i in range(0, len(skills_list)):
        if skills_list[i] in mydict.item().keys():
            new_list.append(mydict.item().get(skills_list[i]))
        else:
            new_list.append(skills_list[i])
    return new_list

In [ ]:
def retrieve_job_title(skills, vectorizer, model, data, nlp, mydict):

    skills_list = pd.Series(skills[0].split(",")).apply(lambda x: x.strip()).tolist() # list sfw skills
    skills_listed = replace_job_portal_skills(skills_list, mydict) # replaces and returns a list
    skills = " ".join(skills_listed)
    
    transformed_skills = vectorizer.transform(pd.Series(skills)) # unlist sfw skills
    probs = model.predict_proba(transformed_skills)
    best_n = np.argsort(probs, axis=1)[:,-4:]
    labels = get_labels(data, 'Sector')
    industries = labels.iloc[best_n[0]]['Sector'].to_list()
    #print(industries)
    all_jobs = data[(data['Sector']== industries[0]) | (data['Sector']== industries[1]) | (data['Sector']== industries[2]) | (data['Sector']== industries[3])]
    
    cf = nlp(str(skills))
    #print("User Input (job portal skills):")
    #print(cf)

    scores = []
    
    
    for j in range(0, len(all_jobs)):
        sfw = nlp(str(all_jobs.iloc[j]['Skill Title']))
        scores.append(sfw.similarity(cf))
      
        
    ind1, ind2, ind3, ind4, ind5 = heapq.nlargest(5, zip(scores, all_jobs['Job Role']))
    
#     print(ind1[0])
#     print(type(ind1[0]))
    if ((ind1[0]<= 0.5) | (ind2[0]<= 0.5) | (ind3[0]<= 0.5)):
        print('Warning: Not a close match')
    

    
    print("SFW Job Role: ", clear_list(ind1[1]), "Score: ", ind1[0])
    print("SFW Job Role: ", clear_list(ind2[1]), "Score: ", ind2[0])
    print("SFW Job Role: ", clear_list(ind3[1]), "Score: ", ind3[0])

In [ ]:
cf_model.iloc[0]

In [ ]:
retrieve_job_title(pd.Series(cf_model.iloc[0]['Skills_unlisted']), vectorizer, model, sf_sectors, nlp, mydict)

In [ ]:
cf_model.iloc[2000]

In [ ]:
cf_model.iloc[584]['Skills_unlisted']

In [ ]:
retrieve_job_title(pd.Series(cf_model.iloc[584]['Skills_unlisted']), vectorizer, model, sf_sectors, nlp, mydict)

In [ ]:
#industry
retrieve_job_title(pd.Series(cf_model.iloc[2000]['Skills_unlisted']), vectorizer, model, sf_sectors, nlp)
#need to use skills 
#not the same sector but close job title from what it seems like based on jd

In [ ]:
#model
retrieve_job_title(pd.Series(cf_model.iloc[2000]['Skills_unlisted']), vectorizer, model, sf_sectors, nlp, mydict)
#need to use skills 
#not the same sector but close job title from what it seems like based on jd

In [ ]:

cf_model.iloc[901]

In [ ]:
#industry
retrieve_job_title(pd.Series(cf_model.iloc[901]['Skills_unlisted']), vectorizer, model, sf_sectors, nlp, mydict)
#need to use skills 
#not the same sector but close job title from what it seems like based on jd

In [ ]:
cf_model.iloc[702]

In [ ]:
retrieve_job_title(pd.Series(cf_model.iloc[702]['Skills_unlisted']), vectorizer, model, sf_sectors, nlp, mydict)
#need to use skills 
#not the same sector but close job title from what it seems like based on jd

In [ ]:
cf_model.iloc[200]

In [ ]:
retrieve_job_title(pd.Series(cf_model.iloc[200]['Skills_unlisted']), vectorizer, model, sf_sectors, nlp, mydict)

In [ ]:
sf_sectors['Sector'].unique()

In [ ]:
closest_1_sfw_jobs = []
closest_1_sfw_skills = []
closest_1_scores = []

closest_2_sfw_jobs = []
closest_2_sfw_skills = []
closest_2_scores = []

closest_2_sfw_jobs = []
closest_2_sfw_skills = []
closest_2_scores = []

job_portal_job = []

for i in range(0, len(cf_model)):
    skills = cf_model['Skills_unlisted'][i]
    data = sf_sector
    top_1_job, top_1_skills, top_1_score, top_2_job, top_2_skills, top_2_score, top_3_job, top_3_skills, top_3_score = retrieve_job_title()

In [ ]:
def retrieve_job_title(skills, vectorizer, model, data, nlp, mydict):

    skills_list = pd.Series(skills[0].split(",")).apply(lambda x: x.strip()).tolist() # list of skills
    skills_listed = replace_job_portal_skills(skills_list, mydict) #replaces and returns a list
    skills = " ".join(skills_listed) #unlistes skills
    
    transformed_skills = vectorizer.transform(pd.Series(skills)) 
    probs = model.predict_proba(transformed_skills)
    best_n = np.argsort(probs, axis=1)[:,-3:]
    labels = get_labels(data, 'Sector')
    industries = labels.iloc[best_n[0]]['Sector'].to_list()
    print(industries)
    all_jobs = data[(data['Sector']== industries[0]) | (data['Sector']== industries[1]) | (data['Sector']== industries[2])]
    
    cf = nlp(str(skills))
    print("User Input (job portal skills):")
    print(cf)

    scores = []
    for j in range(0, len(all_jobs)):
        sfw = nlp(str(all_jobs.iloc[j]['Skill Title']))
        scores.append(sfw.similarity(cf))
      
        
    ind1, ind2, ind3, ind4, ind5 = heapq.nlargest(5, range(len(scores)))
    
    top_1_job = all_jobs.iloc[ind1]['job_role_replaced'] # return actual sfw job
    top_1_skills = all_jobs.iloc[ind1]['Skill Title']  # create a comma separated col of skills in sfw data
    top_1_score = scores[ind1]
    
    top_2_job = all_jobs.iloc[ind2]['job_role_replaced']
    top_2_skills = all_jobs.iloc[ind2]['Skill Title']
    top_2_score = scores[ind2]
    
    top_3_job = all_jobs.iloc[ind3]['job_role_replaced']
    top_3_skills = all_jobs.iloc[ind3]['Skill Title']
    top_3_score = scores[ind3]
    
    return top_1_job, top_1_skills, top_1_score, top_2_job, top_2_skills, top_2_score, top_3_job, top_3_skills, top_3_score
                                                  
    